In [1]:
%reload_ext autoreload
%autoreload 2

import sqlite3
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from utils.dummy import dummy_data
from utils.dummy import dummy_doi
from utils.dummy import get_dummy_info
from utils.utils import get_citation_doi
from utils.utils import get_paper_citation_pairs
from utils.utils import get_cited_by_doi
from utils.utils import get_fully_connected_network
from acm_api.acm import acm_meta
from evaluation.evaluate import Train_Test_Split_Plain
import matplotlib.pyplot as plt
from cf.cf import CF
from cf.cf_unit import CFU
from cf.cf_pr import CFUPR
from cf.cf_full_hits import CFUH
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

<h2>Processing the input file with doi of papers</h2>

In [2]:
doi_list = []
data = {}
data_citedby = {}

# reading the input file with doi of papers
with open('doi/doi.txt', 'r') as doi_file:
    lines = doi_file.readlines()
    
for line in lines:
    doi_list.append(line.strip())
    
con = sqlite3.connect('fake.db')
cur = con.cursor()

is_table_exists = cur.execute(
    """
    SELECT name FROM sqlite_master WHERE type='table'
    AND name='PAPERS'; """).fetchall()

if is_table_exists == []:
    cur.execute(""" 
        CREATE TABLE PAPERS (doi TEXT, data TEXT);
    """)
    
print('processing doi....')
for doi in tqdm(doi_list):
    print(doi)
    cursor = con.execute("SELECT * FROM PAPERS WHERE doi=?;", (doi,))
    fetch_data = cursor.fetchone()
    
    if fetch_data is None:
        temp = acm_meta(doi)
        insert_statement = 'INSERT INTO PAPERS(doi, data) VALUES(?, ?)'
        
        con.execute(insert_statement, (doi, str(temp)))
        data[doi] = get_citation_doi(temp)
        
    else:
        data[doi] = get_citation_doi(fetch_data[1])
        
        for citedby_doi in get_cited_by_doi(fetch_data[1]):
            if not data_citedby.get(citedby_doi, False):
                data_citedby[citedby_doi] = [doi]
            else:
                # A paper can cite my papers in the query set
                data_citedby[citedby_doi].append(doi)
        
con.commit()
con.close()

processing doi....


  0%|          | 0/20 [00:00<?, ?it/s]

1
extracting citation's doi....


  0%|          | 0/100 [00:00<?, ?it/s]

extracting cited_by doi....


...

  0%|          | 0/50 [00:00<?, ?it/s]

20
extracting citation's doi....


  0%|          | 0/100 [00:00<?, ?it/s]

extracting cited_by doi....


  0%|          | 0/50 [00:00<?, ?it/s]

In [3]:
# Paper's doi and it's citations
# data

In [4]:
# data_citedby

<h2>Code for creating the rating matrix</h2>

In [5]:
# Store the citation space
citation_set = set()

# Store the citation space including the cited_by space
citation_set_with_cited_by = set()

# Here we are capturing the all the citation for a given paper
# Later we can use this to know the citation space
for key in data.keys():
    citation_set.update(data[key])
    citation_set_with_cited_by.update(data[key])
    
# We also need to include the queryset itself because they are cited by other papers    
citation_set_with_cited_by.update(data.keys())

# Index of the data without the cited_by key
index_ = data.keys()
                                                   
# Index of the data including the cited_by key
index_with_citedby = set(data.keys())
index_with_citedby.update(data_citedby.keys())
index_with_citedby = list(index_with_citedby)

# This dataframe does not include the cited_by data
df = pd.DataFrame(np.zeros((len(index_), len(citation_set))), index=index_, columns=citation_set)

# This dataframe includes the cited_by data
df_with_cited_by = pd.DataFrame(np.zeros((len(index_with_citedby), len(citation_set_with_cited_by))), \
                                index=index_with_citedby, columns=citation_set_with_cited_by)

for i in index_:
    for j in citation_set:
        if j in data[i]:
            df.loc[i][j] = 1


            
# Here data_citedby is the key-value with key being the paper
# that cites our query set and value is one of the paper in the
# query set
# Similary in data, the key is each paper in query set and
# value is list of citations
overall_ = {**data_citedby, **data}
overall = get_fully_connected_network(overall_, data_citedby.keys(), acm=False)
for i in index_with_citedby:
    for j in citation_set_with_cited_by:
        if j in overall[i]:
            df_with_cited_by.loc[i][j] = 1

df.to_csv('citation-web/matrix-way/citation-matrix.csv')
df_with_cited_by.to_csv('citation-web/matrix-way/citation-full-matrix.csv')

  0%|          | 0/220 [00:00<?, ?it/s]

<h2>Assign root nodes that can be used for cf full version</h2>

In [6]:
root_nodes = df.index.values

<h2>Creating the citation pair</h2>

In [7]:
# This only includes the citation data
citation_pair_path = get_paper_citation_pairs(df)

# This also includes the cited_by data 
citation_pair_full_path = get_paper_citation_pairs(df_with_cited_by, out_file='citation-full-pairs.txt')

writing citation pairs....


  0%|          | 0/20 [00:00<?, ?it/s]

writing citation pairs....


  0%|          | 0/220 [00:00<?, ?it/s]

<h2>Below are the experiments</h2>

In [8]:
evaluate = Train_Test_Split_Plain(CFU, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/20 [00:00<?, ?it/s]

removing the indexes with no citations....
normalizing dataframe....


100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 2838.50it/s]


creating similarity matrix....


...

computing recommendations for : 1


0it [00:00, ?it/s]


finishing evaluation....


  0%|          | 0/10 [00:00<?, ?it/s]

0.0

In [9]:
evaluate = Train_Test_Split_Plain(CF, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/20 [00:00<?, ?it/s]

Creating similarity matrix....


100%|█████████████████████████████████████████| 109/109 [00:01<00:00, 77.58it/s]


Computing recommendations for : 9


...

Computing recommendations for : 1


0it [00:00, ?it/s]


finishing evaluation....


  0%|          | 0/10 [00:00<?, ?it/s]

0.0

In [10]:
evaluate = Train_Test_Split_Plain(CFUPR, min_ref_limit=5, train_size=0.5, normalize_similarity=True)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/20 [00:00<?, ?it/s]

removing the indexes with no citations....
page rank normalization on dataframe....


100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 5485.62it/s]


creating similarity matrix....


...

Computing recommendations for : 1


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 179.87it/s]

finishing evaluation....


  0%|          | 0/10 [00:00<?, ?it/s]

0.2995631807125634

In [14]:
evaluate = Train_Test_Split_Plain(CFUH, min_ref_limit=5, train_size=0.5, normalize_similarity=True, root_node_indexes=root_nodes)
overall = {**data, **data_citedby}
evaluate.fit(overall)

The following doi's are ignored because of low reference :
{'204': 4,
 '205': 3,
 '208': 3,
 '212': 3,
 '223': 4,
 '228': 4,
 '238': 4,
 '242': 4,
 '265': 3,
 '266': 4,
 '277': 4,
 '279': 4,
 '290': 4}
writing citation pairs....


  0%|          | 0/207 [00:00<?, ?it/s]

removing the indexes with no citations....
Deleting :  298
Deleting :  289
Deleting :  218
Deleting :  262
Deleting :  256
Deleting :  284
Deleting :  255
Deleting :  280
Deleting :  251
Deleting :  231
page rank normalization on dataframe....


100%|███████████████████████████████████████| 197/197 [00:00<00:00, 5548.69it/s]


creating similarity matrix....


...

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '432' '488' '486' '484' '419' '438' '454' '418' '410'
 '490' '477' '423' '412' '442' '406' '460' '447' '437' '449' '482' '499'
 '448' '408' '453' '497' '461' '434' '491' '498' '480' '471' '422' '407'
 '439' '405' '462' '445' '466' '436' '473' '409' '451' '468' '430' '431'
 '464' '470' '400' '426' '416' '493' '203' '200' '202' '495' '443' '248'
 '209' '297' '226' '245' '414' '234' '269' '479' '273' '210' '270' '452'
 '283' '217' '403' '465' '292' '278' '230' '249' '216' '220' '429' '207'
 '219' '233' '411' '250' '469' '232' '474' '281' '254' '287' '247' '263'
 '417' '288' '299' '296' '475' '494' '458' '275' '222' '404' '476' '467'
 '463' '291' '496' '450' '295' '487' '440' '481' '261' '294' '472' '267'
 '227' '415' '206' '241' '489' '224' '214' '282' '243' '244' '215' '235'
 '246' '274' '253' '402' '229' '433' '420' '435' '268' '264' '259' '444'
 '272' '258' '237' '485' '211' '285' '401' '201' '

100%|██████████████████████████████████████████| 13/13 [00:00<00:00, 189.18it/s]

finishing evaluation....


  0%|          | 0/94 [00:00<?, ?it/s]

0.21366380205976676

In [4]:
def __tfidf_obj__(sentence_dict):
	"""
	This method is used to generate the objects of countvectorizer
	and tfidfvectorizer
	"""
	countvectorizer = CountVectorizer(analyzer='word', stop_words='english')
	tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words='english')

	# convert th documents into a matrix
	countvectorizer.fit(sentence_dict.values())
	tfidfvectorizer.fit(sentence_dict.values())

	return countvectorizer, tfidfvectorizer


def __tfidf__(sentence_dict):
	"""
	This method is used to process the dictonary of
	sentence into a tf and idf format dataframe
	"""
	countvectorizer, tfidfvectorizer = __tfidf_obj__(sentence_dict)

	# transform with the dataset
	count_wm = countvectorizer.transform(sentence_dict.values())
	tfidf_wm = tfidfvectorizer.transform(sentence_dict.values())

	# retrieve the terms found in the corpora
	# if we take same parameters on both Classes(CountVectorizer and TfidfVectorizer) , 
	# it will give same output of get_feature_names() methods)
	count_tokens = countvectorizer.get_feature_names()
	tfidf_tokens = tfidfvectorizer.get_feature_names()

	df_countvect = pd.DataFrame(data=count_wm.toarray(),
					index=sentence_dict.keys(), columns = count_tokens)

	df_tfidfvect = pd.DataFrame(data=tfidf_wm.toarray(),
					index=sentence_dict.keys(), columns = tfidf_tokens)

	return df_countvect, df_tfidfvect

In [15]:
documents = {
   
    'doc2' : "more mathematical concept inside how is it working than other vectorizer algorithm. Because i think it is very important for tuning performance on NLP projects",
    'doc3' : "This is very common algorithm to transform text into a meaningful representation of numbers which is used to fit machine algorithm for prediction. Let’s take sample example and explore two different spicy sparse matrix before go into deep explanation . It gives overall view what i am trying to explain below"
}



In [13]:
def __tfidf_obj__(sentence_dict):
	"""
	This method is used to generate the objects of countvectorizer
	and tfidfvectorizer
	"""
	countvectorizer = CountVectorizer(analyzer='word', stop_words='english')
	tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words='english')

	# convert th documents into a matrix
	countvectorizer.fit(sentence_dict.values())
	tfidfvectorizer.fit(sentence_dict.values())

	return countvectorizer, tfidfvectorizer


def __tfidf__(sentence_dict):
	"""
	This method is used to process the dictonary of
	sentence into a tf and idf format dataframe
	"""
	countvectorizer, tfidfvectorizer = __tfidf_obj__(sentence_dict)

	# transform with the dataset
	count_wm = countvectorizer.transform(sentence_dict.values())
	tfidf_wm = tfidfvectorizer.transform(sentence_dict.values())

	# retrieve the terms found in the corpora
	# if we take same parameters on both Classes(CountVectorizer and TfidfVectorizer) , 
	# it will give same output of get_feature_names() methods)
	count_tokens = countvectorizer.get_feature_names()
	tfidf_tokens = tfidfvectorizer.get_feature_names()

	df_countvect = pd.DataFrame(data=count_wm.toarray(),
					index=sentence_dict.keys(), columns = count_tokens)

	df_tfidfvect = pd.DataFrame(data=tfidf_wm.toarray(),
					index=sentence_dict.keys(), columns = tfidf_tokens)

	return df_countvect, df_tfidfvect

countvectorizer, tfidfvectorizer = __tfidf_obj__({'doc1' : documents['doc1']})

In [16]:
countvectorizer.transform(documents)

<2x16 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [19]:
countvectorizer.transform(documents).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [20]:
temp = pd.DataFrame([[1,1,2,3], [2,3,3,4], [2,3,4,2], [2,2,3,2]],index=['doc1', 'doc2', 'doc3', 'doc4'], columns=['a','b','c','d'])

In [26]:
temp.sum(axis=0)['a']

7

In [23]:
temp

,a,b,c,d
doc1,1,1,2,3
doc2,2,3,3,4
doc3,2,3,4,2
doc4,2,2,3,2
